In [ ]:
import requests
from bs4 import BeautifulSoup
import bs4
import json
import httpx
import time
import lxml
import cohere
from cohere.classify import Example 
import pandas as pd

In [ ]:
cohere_apiKey = '6rIWhaa4vGVQKRnSJ9ClSKkgsxQp8EabiXBLH2zl'
elsevier_apikey = 'ce04ffa20c7ee4560e4ffd4761bbe157'

In [3]:
class paper_parser():
    def __init__(self, query, apikey, count, start, sort, view, timeout):
        self.query = query
        self.apikey = apikey
        self.count = count
        self.start = start
        self.sort = sort
        self.view = view
        self.timeout = timeout
        self.dic = self.search_for_papers(self.query, self.apikey, self.count, self.start, self.sort, self.view)
        self.piis = list(self.dic.values())
        self.paragraphs = []
    def search_for_papers(self, query, apikey, count, start, sort, view):
        url = 'https://api.elsevier.com/content/search/sciencedirect?'
        headers = {'X-ELS-APIKey': apikey, 'Accept': 'application/json'}
        params = {'query': query, 'count': count, 'start': start, 'sort': sort, 'view': view}
        r = requests.get(url, headers=headers, params=params)
        json_resp = json.loads(r.text)
        titles, piis = [], []
        dic = {}
        for paper in json_resp['search-results']['entry']:
            titles.append(paper['dc:title'])
            piis.append(paper['pii'])
            dic[paper['dc:title']] = paper['pii']
        return dic
    def get_paragraphs(self, pii, apikey, timeout):
        url = f'https://api.elsevier.com/content/article/pii/{pii}'
        response = requests.get(url, headers={"X-ELS-APIKey":apikey, "content-type": "text/xml;charset=UTF-8"}, timeout = timeout)
        soup = BeautifulSoup(response.content, "xml")
        text =[''.join(s.findAll(text=True)).replace("\u202f", "").replace("\n", "") for s in soup.find_all('ce:para')]
        return text
    def get_all_paragraphs(self):
        for pii in self.piis:
            self.paragraphs.append(self.get_paragraphs(pii, self.apikey, self.timeout))
        return self.paragraphs
    def get_all_titles(self):
        

In [4]:
pp = paper_parser('nips, membrane, polysulfone', elsevier_apikey, 10, 0, 'relevance', 'COMPLETE', 60)
example_papers = pp.get_all_paragraphs()

In [5]:
example_papers[0]

['Versatile, facile, easy, and economical, these are the advantages that immersion precipitation or non-solvent phase separation (NIPS) offers [1–3]. Due to these, NIPS process has been regarded as the most widely-used approach in fabricating flat sheet membranes from a variety of polymers [4]. In addition, NIPS can provide a wide-range of pore size: from microfiltration to nanofiltration [5]. In this process, a polymer of choice is dissolved in a suitable solvent. Once homogeneous, the polymeric solution is cast onto a support and phase-inversion is prompted through direct immersion in a coagulation bath/non-solvent bath (typically water), generating a polymer rich phase that transforms into the membrane matrix after polymer precipitation, and a polymer lean phase which develops into pores after demixing [6].',
 'Polysulfone (PSf) membrane is amongst the most extensively used commercial membranes due to its favorable attributes such as high glass transition temperature (Tg\xa0=\xa0190

In [6]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [7]:
Example_data = pd.DataFrame(flatten_list(example_papers), columns = ['paragraph'])
Example_data.to_csv('Example_data.csv')

In [59]:
co = cohere.Client(cohere_apiKey) 
response = co.classify( 
  model='large', 
  inputs=["Am I still able to return my order?", "When can I expect my package?"], 
  examples=[Example("Do you offer same day shipping?", "Shipping and handling policy"), Example("Can you ship to Italy?", "Shipping and handling policy"), Example("How long does shipping take?", "Shipping and handling policy"), Example("Can I buy online and pick up in store?", "Shipping and handling policy"), Example("What are your shipping options?", "Shipping and handling policy"), Example("My order arrived damaged, can I get a refund?", "Start return or exchange"), Example("You sent me the wrong item", "Start return or exchange"), Example("I want to exchange my item for another colour", "Start return or exchange"), Example("I ordered something and it wasn’t what I expected. Can I return it?", "Start return or exchange"), Example("What’s your return policy?", "Start return or exchange"), Example("Where’s my package?", "Track order"), Example("When will my order arrive?", "Track order"), Example("What’s my shipping number?", "Track order"), Example("Which carrier is my package with?", "Track order"), Example("Is my package delayed?", "Track order")]) 
print('The confidence levels of the labels are: {}'.format(response.classifications)) 

The confidence levels of the labels are: [Classification<prediction: "Start return or exchange", confidence: 0.9938701>, Classification<prediction: "Track order", confidence: 0.995854>]
